### Data Preprocessing

Chosen Data Sets:

GoiEner Data Set

Smart Grid Smart City Customer Trial Data Set

METER UK Household Electricity and Activity Survey, 2016-2019

Original SmartMeter dataset (year 2014)

Apart from electricity use, there are some other interesting information in various csv files.

There is a seperate CSV file for Type of  Home Area Network appliance or room electricity usage readings. This can be a subject of different study or it can be included in our publication as an additional analysis. There is a unique customer ID and names of different home appliances / home areas and these can be used for a federated network (Considering each home appliance / home area as a node in the federated framework), by labeling each home appliance / home area with a number.

Apart from this, there are csv files containing information about households (demographics, lifestyle and etc.). This can be used as additional features to improve forecasting (May be the algorithms / models employed will learn some patterns from these, people with certain lifestyle, age group, location etc behaving in similar patterns etc.) Also, this can be used to assess fairness aspects of the models.

Lastly there are 3 CSV files for offers, peak events, and peak event responses. Consider also using these information to improve forecasting.



Install Dependencies

In [2]:
import dask
import dask.dataframe as dd
import holidays


In [3]:
dask.config.set(temporary_directory='D:/Dask')

In [4]:
# First let's load the data set.

file_path_sgsc = 'D:/FL Publication/Datasets for the Publication/Australia - Smart-Grid Smart-City Customer Trial Data/electricity_use_interval_readings/electricity_use_interval_readings.csv'
sgsc_data = dd.read_csv(file_path_sgsc, assume_missing=True)

In [5]:
print(sgsc_data.columns)

Index(['CUSTOMER_ID', 'READING_DATETIME', ' CALENDAR_KEY', ' EVENT_KEY',
       ' GENERAL_SUPPLY_KWH', ' CONTROLLED_LOAD_KWH', ' GROSS_GENERATION_KWH',
       ' NET_GENERATION_KWH', ' OTHER_KWH'],
      dtype='object')


In [ ]:
# First check number of rows to make sure that data is complete.
# Get the number of rows and columns in the Dask DataFrame
num_rows, num_cols = sgsc_data.shape
# Compute the number of rows
num_rows = sgsc_data.shape[0].compute()
print(f"Number of rows: {num_rows}")

print(f"Number of columns: {num_cols}")

In [6]:
# We will remove some of the columns that we do not need for our first analysis.

# For now we will not consider peak events,so we do not need Event_Key column

#Since much of the PV generation and controlled tariff is 0 we will ignore these columns as well and focus our attention on general supply.

columns_to_drop = [' EVENT_KEY', ' CONTROLLED_LOAD_KWH', ' GROSS_GENERATION_KWH', ' NET_GENERATION_KWH', ' OTHER_KWH']
sgsc_data = sgsc_data.drop(columns_to_drop, axis=1)

In [7]:
sgsc_data['READING_DATETIME'] = dd.to_datetime(sgsc_data['READING_DATETIME'])

In [8]:
# Display initial data types
print("Original data types:\n", sgsc_data.dtypes)

Original data types:
 CUSTOMER_ID                   float64
READING_DATETIME       datetime64[ns]
 CALENDAR_KEY                 float64
 GENERAL_SUPPLY_KWH           float64
dtype: object


In [9]:
# Extract datetime components
sgsc_data['year'] = sgsc_data['READING_DATETIME'].dt.year.astype('int16')
sgsc_data['month'] = sgsc_data['READING_DATETIME'].dt.month.astype('int8')
sgsc_data['day'] = sgsc_data['READING_DATETIME'].dt.day.astype('int8')
sgsc_data['hour'] = sgsc_data['READING_DATETIME'].dt.hour.astype('int8')
sgsc_data['minute'] = sgsc_data['READING_DATETIME'].dt.minute.astype('int8')
sgsc_data['dayofyear'] = sgsc_data['READING_DATETIME'].dt.dayofyear.astype('int16')
sgsc_data['dayofweek'] = sgsc_data['READING_DATETIME'].dt.dayofweek.astype('int8')
sgsc_data['is_weekend'] = sgsc_data['dayofweek'].isin([5, 6]).astype('bool')
sgsc_data[' GENERAL_SUPPLY_KWH'] = sgsc_data[' GENERAL_SUPPLY_KWH'].astype('float32')

In [10]:
# Create holidays column
au_holidays = holidays.Australia()

# Since we're using Dask, we need to create a function to check for holidays
def is_holiday(date):
    return date in au_holidays

# Add is_holiday column using map_partitions
sgsc_data['is_holiday'] = sgsc_data['READING_DATETIME'].map_partitions(lambda x: x.map(is_holiday))

In [ ]:
# Compute the total number of rows
total_rows = len(sgsc_data.compute())
print(f"Total number of rows in the dataset: {total_rows}")

# Display the first few rows to verify the new columns
print("\nFirst few rows of the processed dataset:")
print(sgsc_data.head())

In [11]:
sgsc_data['is_holiday'] = sgsc_data['is_holiday'].astype('bool')

In [12]:
print("\nOptimized data types:\n", sgsc_data.dtypes)


Optimized data types:
 CUSTOMER_ID                   float64
READING_DATETIME       datetime64[ns]
 CALENDAR_KEY                 float64
 GENERAL_SUPPLY_KWH           float32
year                            int16
month                            int8
day                              int8
hour                             int8
minute                           int8
dayofyear                       int16
dayofweek                        int8
is_weekend                       bool
is_holiday                       bool
dtype: object


In [13]:
# Drop the original READING_DATETIME column
sgsc_data = sgsc_data.drop('READING_DATETIME', axis=1)

In [14]:
print(sgsc_data.columns)

Index(['CUSTOMER_ID', ' CALENDAR_KEY', ' GENERAL_SUPPLY_KWH', 'year', 'month',
       'day', 'hour', 'minute', 'dayofyear', 'dayofweek', 'is_weekend',
       'is_holiday'],
      dtype='object')


In [ ]:
# Check for missing values

missing_customer_id = sgsc_data['CUSTOMER_ID'].isnull().sum().compute()
missing_general_supply_kwh = sgsc_data[' GENERAL_SUPPLY_KWH'].isnull().sum().compute()
missing_kalender_key = sgsc_data[' CALENDAR_KEY'].isnull().sum().compute()


print(f"Missing values in CUSTOMER_ID: {missing_customer_id}")
print(f"Missing values in GENERAL_SUPPLY_KWH: {missing_general_supply_kwh}")
print(f"Missing values in CUSTOMER_ID: {missing_kalender_key}")

# There is no missing values.

In [15]:
def aggregate_to_hourly(df):
    """
    Aggregate half-hourly energy consumption data to hourly by summing up the energy consumption
    for each customer for each hour.
    
    Parameters:
    df (dask.dataframe.DataFrame): Input DataFrame with half-hourly data
    
    Returns:
    dask.dataframe.DataFrame: Aggregated hourly data
    """
    # Group by all relevant columns except minute and CALENDAR_KEY
    grouped = df.groupby([
        'CUSTOMER_ID',
        'year',
        'month',
        'day',
        'hour',
        'dayofyear',
        'dayofweek',
        'is_weekend',
        'is_holiday'
    ])
    
    # Sum up the energy consumption
    aggregated = grouped.agg({
        ' GENERAL_SUPPLY_KWH': 'sum'
    }).reset_index()
    
    return aggregated

In [16]:
hourly_data = aggregate_to_hourly(sgsc_data)

In [17]:
hourly_data = hourly_data.sort_values(
    by=['CUSTOMER_ID', 'year', 'month', 'day', 'hour']
).reset_index(drop=True)

In [18]:
hourly_data.to_csv('D:/FL Publication/Datasets for the Publication/Australia - Smart-Grid Smart-City Customer Trial Data/electricity_use_interval_readings/electricity_use_hourly.csv', single_file=True, index=False)

['D:\\FL Publication\\Datasets for the Publication\\Australia - Smart-Grid Smart-City Customer Trial Data\\electricity_use_interval_readings\\electricity_use_hourly.csv']